## Before you run:
    
    1. Please make sure you have stable internet connection.
    2. Code runs for a long time please do not let your PC to sleep.
  

### Part I
1.	Your code should initialize a Chrome web driver and access the Wallapop webpage
2.	From here, it should use the searchbox to look for all results matching the keyword “bicicleta” (bike in Spanish)
3.	Under the different search options provided, your code should only access those entries in the category “Bicicletas” (bikes in Spanish)
4.	Filter the search results according to the following criteria:
        - Location. You should only retrieve results in “España, Barcelona” and narrow down the search to a maximum of “10km”
        - Price. Because you will probably be looking for second hand goods, limit the price to 800€.
        - Subfield. Within the “Bicicletas” field, there are multiple subfields available. Narrow down your search to include only results in the “Bicicletas y triciclos” subfield. 
        - Labels. You can further narrow down your search by choosing more refined subfields. You should retrieve only those results that correspond to “Bicicletas de carretera” (road bikes), “Bicicletas plegables” (foldable bikes) or “MTB” (mountain bikes).
        - State. You are only interested in bikes that are reasonably new. Hence, you should only include those results that correspond to bikes that are “Nuevo” (New), “Como nuevo” (As good as new) and “En buen estado” (In good condition).
5.	Considering all the different combinations among the options above ("Bicicletas de carretera" and "Nuevo", "Bicicletas de carretera" and "En buen estado", etc.), retrieve the following information for all the available results (if less than 250) or for the first 250 results (if more than 250):Exclude all results corresponding to adds.
        - The URL address to the post
        - The URL address to the displayed image
        - The title of the post
        - The price
        - The full description (as shown in the results page)
6.	Store the retrieved information in a DataFrame called df under the following columns:
        - "Link" (in str form)
        - “Title” (in str/object form)
        - “Description” (in str/object form)
        - “Price” (in float form)
        - “Image” (in str/object form).


In [ ]:
# import relevant libraries
import requests as rq
import bs4
import time
import selenium
import os
import itertools
import re
import pandas as pd
import numpy as np
import tqdm
from datetime import datetime

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains


# define own formulas 
def common(a,b): 
    ''' 
    Check for common values in 2 lists 
    '''
    c = [value for value in a if value in b] 
    if len(c) > 0:
        return True
    else:
        return False


In [ ]:
# Go to Wallapop Page and accept cookies
path = '/Selenium/chromedriver.exe'
driver = webdriver.Chrome(executable_path = path)
driver.get("https://es.wallapop.com/")
time.sleep(2)
driver.maximize_window()
try:
    cookies_accept = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, r'//*[@id="onetrust-accept-btn-handler"]')))
    cookies_accept.click()
except:
    pass

# Initialize Search
try:
    search_bar = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, r'//*[@id="searchBoxForm"]/div/div[1]/input[1]')))
    search_bar.send_keys('bicicleta')
    click_result = WebDriverWait(driver, timeout=15).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="searchBoxForm"]/div/div[3]/ul/li[1]/span[1]')))
    click_result.click()
except:
    pass

# Set Location

apply_filter = WebDriverWait(driver, timeout=15).until(EC.element_to_be_clickable((By.XPATH, '/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/div/div/tsl-bubble/div/div/div')))
apply_filter.click()
time.sleep(2)
set_location = WebDriverWait(driver, timeout=15).until(EC.element_to_be_clickable((By.XPATH, '/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/tsl-drawer/div[1]/div[1]/tsl-filter-group/div/div[5]/tsl-filter-host/div/tsl-location-filter/tsl-filter-template/div/tsl-drawer-placeholder-template/tsl-select-parent-option/div/tsl-select-option/div/div/span')))
set_location.click()
time.sleep(2)
donde_location = WebDriverWait(driver, timeout=15).until(EC.element_to_be_clickable((By.XPATH, '/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/tsl-drawer/div[1]/div[1]/tsl-filter-group/div/div[5]/tsl-filter-host/div/tsl-location-filter/tsl-filter-template/div/tsl-drawer-placeholder-template/div/div[2]/div/div[1]/input')))
donde_location.click()
donde_location.clear()
time.sleep(2)

#write - España, Barcelona
donde_location.send_keys('España, Barcelona')
time.sleep(2)

# Choose 1st option
choose_BCN = WebDriverWait(driver, timeout=15).until(EC.element_to_be_clickable((By.XPATH, '/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/tsl-drawer/div[1]/div[1]/tsl-filter-group/div/div[5]/tsl-filter-host/div/tsl-location-filter/tsl-filter-template/div/tsl-drawer-placeholder-template/div/div[2]/div/div[1]/ngb-typeahead-window/button[1]/div/span')))
choose_BCN.click()
time.sleep(2)

# Slider in location

slider_XPATH = '/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/tsl-drawer/div[1]/div[1]/tsl-filter-group/div/div[5]/tsl-filter-host/div/tsl-location-filter/tsl-filter-template/div/tsl-drawer-placeholder-template/div/div[2]/div/form/tsl-slider-form/form/ngx-slider/span[5]'
slider = WebDriverWait(driver, timeout=15).until(EC.element_to_be_clickable((By.XPATH,slider_XPATH)))

# Reset slider to 1 KM
i= 0
while i < 10:
    slider.send_keys(Keys.LEFT)
    i = i+1
i= 0
#Set Slider:  1 for 5 KM | 2 for 10 KM | 3 for 30 KM| 4 for 50 KM| 5 for 100 KM | 6 for 200 KM | 7 for Max KM 
while i < 2:
    slider.send_keys(Keys.RIGHT)
    i= i + 1
i= 0
time.sleep(2)
aplicar = WebDriverWait(driver, timeout=15).until(EC.element_to_be_clickable((By.XPATH, '/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/tsl-drawer/div[1]/div[1]/tsl-filter-group/div/div[5]/tsl-filter-host/div/tsl-location-filter/tsl-filter-template/div/tsl-drawer-placeholder-template/div/div[3]/tsl-button/button')))
aplicar.click()
time.sleep(2)

#Set max price to 800:
set_price = WebDriverWait(driver, timeout=15).until(EC.element_to_be_clickable((By.XPATH, '/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/tsl-drawer/div[1]/div[1]/tsl-filter-group/div/div[2]/tsl-filter-host/div/tsl-range-filter/tsl-filter-template/div/form/div/div[2]/input')))
set_price.clear()
set_price.click()
time.sleep(2)
set_price.send_keys('800')
time.sleep(2)


#Final aplicar
aplicar = WebDriverWait(driver, timeout=15).until(EC.element_to_be_clickable((By.XPATH, '/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/tsl-drawer/div[1]/div[2]/tsl-button[2]/button')))
aplicar.click()

### Part II 
7.	In addition to the above, include an additional column “Type” including a reference to whether the considered record corresponds to a “Bicicleta de carretera”, a “Bicicleta plegable” or “MTB”
8.	Also include a column “State” to specify whether the considered bike is “Nuevo”, “Como nuevo” or “En buen estado”.
9.	It is possible that the same result is included more than once in the different searches. To avoid redundancy, drop all the duplicates from your DataFrame
10.	Include an additional column “Children” to identify whether the considered entry corresponds to a children bike or not. These should be identified by the keywords “niño/a”, “niño”, “niña”, “niños” , “niñas” or "niño/as", appearing either in the title or in the description of each result. Fill in the “Children” column with a boolean True whenever each of the provided keywords are present.
11.	An important piece of information is the size of the bike. Not all search results will specify the size. When they do, make sure you retrieve these data and store them in a new column “Size” (in int form). The keyword for size in Spanish is “talla”. Note that bike sizes are usually along the ranges 54 - 60 approx.
12.	Sometimes the size will be specified as S, M or L, When this happens, store the corresponding information in a new column called "Size (letter)"
13.	Write the code to create a new dataframe called agg that includes the aggregated data for the "Type" and "State" columns. This Dataframe should include the average prices by "Type" and "State" (in this order)


In [ ]:
# Creation of lists for scraping
# Main lists
images = [] 
titles = [] 
prices = [] 
description = [] 
types = [] 
states = [] 
links = [] 
pub_dates = [] 
talla = []
size_let = []
children_bike = []


# supporting lists
Talla_S = ['TALLAS']
Talla_M = ['TALLAM']
Talla_L = ['TALLAL']
child_terms = ['NIÑO/A', 'NIÑO', 'NIÑA' , 'NIÑOS' , 'NIÑAS', 'NIÑOS/AS', 'INFANTIL']

dic_es_en={"ene":"Jan",
            "feb":"Feb",
            "mar":"Mar",
            "abr":"Apr",
            "may":"May",
            "jun":"Jun",
            "jul":"Jul",
            "ago":"Aug",
            "sep":"Sep",
            "oct":"Oct",
            "nov":"Nov",
            "dic":"Dec"}

In [ ]:
# Actual Scraping Process

#Filtering for the 9 cases

#Subcategory:
Condition_Types_XPath = ['/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/div/div[2]/tsl-filter-group/div/div[5]/tsl-filter-host/div/tsl-multi-select-filter/div/tsl-filter-template/div/div/div[2]/div/tsl-drawer-placeholder-template/div/div/form/tsl-multi-select-form/div[1]/tsl-multi-select-option/div/tsl-checkbox-form',
                         '/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/div/div[2]/tsl-filter-group/div/div[5]/tsl-filter-host/div/tsl-multi-select-filter/div/tsl-filter-template/div/div/div[2]/div/tsl-drawer-placeholder-template/div/div/form/tsl-multi-select-form/div[2]/tsl-multi-select-option/div/tsl-checkbox-form',
                         '/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/div/div[2]/tsl-filter-group/div/div[5]/tsl-filter-host/div/tsl-multi-select-filter/div/tsl-filter-template/div/div/div[2]/div/tsl-drawer-placeholder-template/div/div/form/tsl-multi-select-form/div[3]/tsl-multi-select-option/div/tsl-checkbox-form']

# list with different categories that we can filter from 
categories = ['Bicicletas de carretera','Bicicletas plegables','MTB']

#loop that selects each category for the combinations in the filtering
for category in categories:
    try:
        driver.switch_to.window(driver.window_handles[0])
                                
        apply_filter = WebDriverWait(driver, timeout=15).until(EC.element_to_be_clickable((By.XPATH, '/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/div/div/tsl-bubble/div/div/div')))
        apply_filter.click() #click in the filter
                    
        subcategory = WebDriverWait(driver, timeout=15).until(EC.element_to_be_clickable((By.XPATH, '/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/tsl-drawer/div[1]/div[1]/tsl-filter-group/div/div[3]/tsl-filter-host/div/tsl-multi-select-filter/div/tsl-filter-template/div/tsl-drawer-placeholder-template/tsl-select-parent-option/div/tsl-select-option/div/div/span')))
        subcategory.click() #click to select subcategory
        
        #"spots" to take into account possible different locations 
        second_spot = WebDriverWait(driver, timeout=15).until(EC.element_to_be_clickable((By.XPATH, '/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/tsl-drawer/div[1]/div[1]/tsl-filter-group/div/div[3]/tsl-filter-host/div/tsl-multi-select-filter/div/tsl-filter-template/div/tsl-drawer-placeholder-template/div/div[2]/div/form/tsl-multi-select-form/div[3]')))
        third_spot = WebDriverWait(driver, timeout=15).until(EC.element_to_be_clickable((By.XPATH, '/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/tsl-drawer/div[1]/div[1]/tsl-filter-group/div/div[3]/tsl-filter-host/div/tsl-multi-select-filter/div/tsl-filter-template/div/tsl-drawer-placeholder-template/div/div[2]/div/form/tsl-multi-select-form/div[2]')))
        if 'Bicicletas y triciclos' in second_spot.text.strip() : 
            second_spot.click()
        elif 'Bicicletas y triciclos' in third_spot.text.strip(): 
            third_spot.click()
    except:
        pass
    
    #look at different options that we can filter from
    different_options = driver.find_elements(By.CSS_SELECTOR,".MultiSelectOption__label.m-0.d-block")
    index = 0
    
    #Find location of Biciletas y Triciletas and select the relevant filters.
    for z in range(len(different_options)): 
        
        driver.switch_to.window(driver.window_handles[0]) # set active instance to search page
        
        try:                
            apply_filter = WebDriverWait(driver, timeout=15).until(EC.element_to_be_clickable((By.XPATH, '/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/div/div/tsl-bubble/div/div/div')))
            apply_filter.click() #click in the filter
    
            subcategory = WebDriverWait(driver, timeout=15).until(EC.element_to_be_clickable((By.XPATH, '/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/tsl-drawer/div[1]/div[1]/tsl-filter-group/div/div[3]/tsl-filter-host/div/tsl-multi-select-filter/div/tsl-filter-template/div/tsl-drawer-placeholder-template/tsl-select-parent-option/div/tsl-select-option/div/div/span')))
            subcategory.click() #click to select subcategory
    
            #"spots" to take into account possible different locations
            second_spot = WebDriverWait(driver, timeout=15).until(EC.element_to_be_clickable((By.XPATH, '/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/tsl-drawer/div[1]/div[1]/tsl-filter-group/div/div[3]/tsl-filter-host/div/tsl-multi-select-filter/div/tsl-filter-template/div/tsl-drawer-placeholder-template/div/div[2]/div/form/tsl-multi-select-form/div[3]')))
            third_spot = WebDriverWait(driver, timeout=15).until(EC.element_to_be_clickable((By.XPATH, '/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/tsl-drawer/div[1]/div[1]/tsl-filter-group/div/div[3]/tsl-filter-host/div/tsl-multi-select-filter/div/tsl-filter-template/div/tsl-drawer-placeholder-template/div/div[2]/div/form/tsl-multi-select-form/div[2]')))
            if 'Bicicletas y triciclos' in second_spot.text.strip() : 
                second_spot.click()
            elif 'Bicicletas y triciclos' in third_spot.text.strip(): 
                third_spot.click()
        except:
            pass
                
            try:
                reset_Xpath = '/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/tsl-drawer/div[1]/div[1]/tsl-filter-group/div/div[3]/tsl-filter-host/div/tsl-multi-select-filter/div/tsl-filter-template/div/tsl-drawer-placeholder-template/div/div[2]/div/form/tsl-multi-select-form/div/div[1]/div/p'
                reset_button = driver.find_element(By.XPATH, reset_Xpath)
                reset_button.click()
            except:
                pass 
        
        different_options = driver.find_elements(By.CSS_SELECTOR,".MultiSelectOption__label.m-0.d-block")
        
        
        if different_options[z].text.strip() == category: #Checks if the different options' text is equal to the desired category
            j = z-9 #subtracts 9 because of website structure 
            number = str(j) #number as a string to add below (in Biciletas_XPath) and click on the desired button
            Biciletas_XPath = str('/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/tsl-drawer/div[1]/div[1]/tsl-filter-group/div/div[3]/tsl-filter-host/div/tsl-multi-select-filter/div/tsl-filter-template/div/tsl-drawer-placeholder-template/div/div[2]/div/form/tsl-multi-select-form/div/div[2]/div['+number+']/tsl-multi-select-option/div/tsl-checkbox-form/label')
            Biciletas_button = WebDriverWait(driver, timeout=15).until(EC.element_to_be_clickable((By.XPATH, Biciletas_XPath)))
            Biciletas_button.click()          
            time.sleep(2)
            
            #return to main page by going back and then clicking on "aplicar"
            go_back = WebDriverWait(driver, timeout=15).until(EC.element_to_be_clickable((By.XPATH, '/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/tsl-drawer/div[1]/div[1]/tsl-filter-group/div/div[3]/tsl-filter-host/div/tsl-multi-select-filter/div/tsl-filter-template/div/tsl-drawer-placeholder-template/div/div[1]/tsl-svg-icon')))
            go_back.click()
            time.sleep(2)
            aplicar = WebDriverWait(driver, timeout=15).until(EC.element_to_be_clickable((By.XPATH, '/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/tsl-drawer/div[1]/div[2]/tsl-button[2]/button')))
            aplicar.click()
                              
            #Conditions - Select 3 conditions 1 type at a time            
            estado_Xpath = '/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/div/div/tsl-filter-group/div/div[5]'
                            
            for t in Condition_Types_XPath:
                            
                driver.switch_to.window(driver.window_handles[0])
                estado_button = driver.find_element(By.XPATH, estado_Xpath)
                estado_button.click()
                            
                time.sleep(1)
                
                try:
                    reset_button_Xpath = '/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/div/div[2]/tsl-filter-group/div/div[5]/tsl-filter-host/div/tsl-multi-select-filter/div/tsl-filter-template/div/div/div[2]/div/tsl-drawer-placeholder-template/div/div/form/tsl-multi-select-form/p'
                    reset_button = driver.find_element(By.XPATH, reset_button_Xpath)
                    reset_button.click()
                except:
                    pass
                
                
                filter_checkbox = driver.find_element(By.XPATH, t)
                filter_checkbox.click()
                time.sleep(1)
                aplicar_Button_Xpath = '/html/body/tsl-root/tsl-public/div/div/tsl-search/div/div/tsl-filters-wrapper/div/div[2]/tsl-filter-group/div/div[5]/tsl-filter-host/div/tsl-multi-select-filter/div/tsl-filter-template/div/div/div[3]/tsl-button[2]'
                aplicar_Button = driver.find_element(By.XPATH, aplicar_Button_Xpath)
                aplicar_Button.click() # click in "aplicar"
                
                # Set Variable to scroll down the page
                first_ad_CSS = 'body'
                first_ad_element = driver.find_element(By.CSS_SELECTOR, first_ad_CSS)
                
                # Scroll Down 50 times to enable 'Ver Mas Products' Button
                i = 0
                while i < 50:
                    first_ad_element.send_keys(Keys.PAGE_DOWN)
                    i = i+1
                i = 0
               
                # Slide up until 'Ver Mas Productos' is visible on the screen
                g = 0
                try:
                    while g < 50:
                        Boolean_result = driver.find_element(By.XPATH,'//button[contains(text(), "Ver más productos")]').is_displayed()
                        if Boolean_result:
                            first_ad_element.send_keys(Keys.PAGE_UP)
                            time.sleep(0.3)
                            driver.find_element(By.XPATH,'//button[contains(text(), "Ver más productos")]').click()
                            g = 51
                        else:
                            g+1
                except:
                    pass
                
                while i < 100:
                    time.sleep(0.3)
                    first_ad_element.send_keys(Keys.PAGE_DOWN)
                    i = i+1
                
                i=0
                
                #Extracting first infos from result overview (only price, image url and title)
                grid_block = driver.find_element(By.XPATH, '//div[@class = "ItemCardList grid-lg-4 grid-md-3 grid-sm-2 grid-xl-4 grid-xs-2 m-auto w-100"]')
                bici_list = grid_block.find_elements(By.XPATH, '//a[@class="ItemCardList__item ng-star-inserted"]')
                
                
                if len(bici_list)<250: 
                    no_of_postings=len(bici_list) # if lenght is less than the one required
                else:
                    no_of_postings=250 # maximum lenght required
                
                
                for bici in range(no_of_postings): 
                    # prices
                    try:
                        prices.append(int(bici_list[bici].find_element(By.TAG_NAME, 'span').text.split('€')[0].strip())) # append price to price's list
                    except:
                        prices.append(None) # in case price is not specified
                    # titles
                    try:
                        title = bici_list[bici].find_element(By.TAG_NAME, 'img').get_attribute('alt') 
                        titles.append(title)
                    except:
                        titles.append(None) # in case there is no title
                    
                    # image urls
                    try:
                        images.append(bici_list[bici].find_element(By.TAG_NAME, 'img').get_attribute('src'))
                    except:
                        images.append(None)
                    
               
                # Scraping of detail page
                r=1
                for bici in range(no_of_postings): 
                    
                    # make sure all detail pages are closed after the loop has run
                    try:
                        driver.switch_to.window(driver.window_handles[1])
                        driver.close()
                    except:
                        pass
                    
                    # go to individual entry/detail page of offer
                    driver.switch_to.window(driver.window_handles[0])
                    item_Card_XPATH = '/html/body/tsl-root/tsl-public/div/div/tsl-search/div/tsl-search-layout/div/div[2]/div/tsl-public-item-card-list/div/a['+str(r)+']'
                    time.sleep(1)
                    items_Element = driver.find_element(By.XPATH, item_Card_XPATH)
                    items_Element.click()
                    time.sleep(1)
                    driver.switch_to.window(driver.window_handles[1])
                    r = r+1
                    time.sleep(0.5)
                    
                    # catch entries that are no longer available. Code will fill remaining variables with None and the skip to the next item in loop
                    try:
                        WebDriverWait(driver, timeout=15).until(EC.element_to_be_clickable((By.XPATH, "//div[@class = 'container-detail clearfix']")))
                        time.sleep(1)
                    except TimeoutException:
                        links.append(driver.current_url)
                        description.append(None)
                        prices.append(None)
                        types.append(None)
                        states.append(None)
                        children_bike.append(None)
                        talla.append(None)
                        size_let.append(None)
                        pub_dates.append(None)
                        
                        driver.close()
                        driver.switch_to.window(driver.window_handles[0])
                        
                        continue
  
                    
                    #scrape the detail page
                    # Links 
                    try:
                        active_url = driver.current_url
                        links.append(active_url) # append link to the links' list
                    except:
                        pass

                    # description
                    try: #try to find description with xpath and appends its text
                        desc = driver.find_element(By.XPATH, "//p[@class = 'js__card-product-detail--description card-product-detail-description']").text.strip()
                        description.append(desc)
                    except:
                        description.append(None) #in case there is no description


                    #states 
                    try:
                        states_list = driver.find_elements(By.CLASS_NAME, r'ExtraInfo__text')
                        for i in range(len(states_list)):
                            if 'Bueno' in states_list[i].text:
                                states.append('En Buen Estado')
                            elif 'Como nuevo' in states_list[i].text:
                                states.append('Como Nuevo')
                            elif 'Nuevo' in states_list[i].text:
                                states.append('Nuevo')
                            else:
                                states.append(None)
                    except:
                        pass

                    # types
                    try:
                        bubble_list = driver.find_elements(By.CLASS_NAME, r'ExtraInfo__taxonomyBubble')
                        # print(bubble_list)
                        for i in bubble_list:
                            if i.text.strip() == 'MTB':
                                types.append('MTB')
                            elif i.text.strip() == 'Bicicletas plegables':
                                types.append('Bicicleta plegable')
                            elif i.text.strip() == 'Bicicletas de carretera':
                                types.append('Bicicleta de carretera')

                    except:
                        types.append(None)



                    # Publication dates 
                    try:
                        date = driver.find_element(By.CLASS_NAME, 'card-product-detail-user-stats-published').text.strip()
                        date_corr = date.split("-")[0]+"-"+dic_es_en[date.split("-")[1]]+"-"+date.split("-")[2]
                        pub_dates.append(datetime.strptime(date_corr, '%d-%b-%Y').strftime('%d/%m/%Y'))
                    except:
                        pub_dates.append(None)


                    #Size in int
                    talla_help_0 = [] #helpful list
                    talla_help_1 = [] #helpful list
                    talla_help_2 = [] #helpful list
                    talla_help_3 = [] #helpful list
                    talla_help_4 = [] #helpful list
                    text_for_talla = WebDriverWait(driver, timeout=15).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.js__card-product-detail--description.card-product-detail-description')))
                    text_for_talla = text_for_talla.text.upper() #put everything in upper case
                    list_for_talla = text_for_talla.split(' ') #split every word
                    length_talla = len(talla) 
                    for i in range(len(list_for_talla)):
                        if list_for_talla[i] == 'TALLA':
                            x = i # x =  position of talla in the list
                    try: #because if there no "talla", x has no value
                        a = x-3
                        b = x+3
                    except:
                        a = b = x = 0
                    if a < 0: #avoid going out of list lenght's range
                        a = 0
                    if b > len(list_for_talla): #avoid going out of list lenght's range
                        b = len(list_for_talla)
                    #Check for numbers near "talla"
                    for k in range(a,b):
                        for character in list_for_talla[k]:
                            try:
                                int(character)
                                talla_help_0.append(list_for_talla[k]) #if character is int, append to talla_help_0
                            except:
                                pass
                    for i in talla_help_0:
                        try:   
                            if int(i) in range(50,61):
                                talla_help_3.append(int(i)) #append numbers to talla_help_3, however, it is going to append it twice
                                talla_help_4.append(talla_help_3[0]) #solve the problem of appending it twice
                                pass
                        except:
                            pass
                    try:
                        talla.append(talla_help_4[0]) #if talla_help_4 has a number then it is the final value of talla
                    except:
                        pass
                    if length_talla == len(talla): #if nothing has been appended
                        try: #because sometimes they say "talla M" -> no numbers so nothing to append
                            talla_help_1.append(talla_help_0[0]) #only appends one because talla_help_0 will come with repetitive numbers
                        except:
                            pass
                        #only numbers in talla 
                        for i in talla_help_1:
                            for character in i:
                                try:
                                    int(character)
                                    talla_help_2.append(character) #if character is int, append to talla_help_2
                                except:
                                    pass
                        if len(talla_help_2) != 0: #if it is zero then append None because no value
                            a = int(''.join(talla_help_2[0:2])) #join both the two first elements of the list (in the case "57Cm" -> code makes it ['5','7'])
                            if int(a) in range(50,61):
                                talla.append(a)
                            else:
                                talla.append(None)
                        else:
                            talla.append(None)


                    # get relevant search areas ready to look for Size in letter form and if it is a bike for children
                    hashtags = driver.find_element(By.XPATH, r'/html/body/div[2]/div[2]/div[2]/div[2]/div[1]/div[3]/div[1]/div[5]').text.strip().upper().split('#')
                    description_list = desc.upper().split()
                    title_words = driver.find_element(By.ID, r'item-detail-title').text.strip().upper().split()

                    # size (S,M,L) 
                    try:
                        # look in hastags
                        if common(Talla_S, hashtags) == True:
                                size_let.append('S')
                        elif common(Talla_M, hashtags) == True:
                                size_let.append('M')
                        elif common(Talla_L, hashtags) == True:
                                size_let.append('L')
                        # look in description
                        elif 'TALLA' in description:
                            x = description.index("TALLA")
                            if len(description[x+1]) == 1:
                                size_let.append(description[x+1])
                            else:
                                size_let.append(None)

                         # look in title
                        elif 'TALLA' in title_words:
                            y = title_words.index("TALLA")
                            if len(title_words[y+1]) == 1:
                                size_let.append(title_words[y+1])
                            else:
                                size_let.append(None)
                        else:
                            size_let.append(None)
                    except:
                        pass


                    # check if bike for children

                    try:
                        # check in title
                        if common(title_words, child_terms) == True:
                            children_bike.append(True)
                        #check hashtags
                        elif common(hashtags, child_terms) == True:
                            children_bike.append(True)
                        # check description
                        elif common(description_list, child_terms) == True:
                            children_bike.append(True)
                        else:
                            children_bike.append(False)
                    except:
                        children_bike.append(False)

                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])

                        
             
            else:
                pass

In [ ]:
#Closing the Selenium web driver
driver.quit()

### Part 3:
#### Create the dataframe with all the lists
Further duplicates will be dropped adn the None values will be replaced with NaN values

In [ ]:
dict_for_df = {'Link': links,
               'Title':titles, 
               'Description': description,
               'Price':prices, 
               'Image':images, 
               'Type': types,
               'State': states,  
               'Children': children_bike,
               'Size': talla,
               'Size (letter)': size_let,
               'Publication Date': pub_dates
               }

df = pd.DataFrame(dict_for_df) 
df.drop_duplicates(keep=False, inplace = True) 
df.replace(to_replace = np.nan, value = None, inplace = True)
df

### Part 4:
#### Creating the aggregated overview of average prices by Type and State

In [ ]:
# price data is aggregated according to Type and State
agg=df.groupby(["Type","State"])["Price"].mean().reset_index(name="Average Price")

# Average price is rounded via a lambda function
agg["Average Price"]=agg["Average Price"].apply(lambda x : round(x))

agg